In [ ]:
!pip install pyfpgrowth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5487 sha256=dde8ec95489e54537dbb4931fa5e3d03a95f08617625b512217bebddfcaea626
  Stored in directory: /root/.cache/pip/wheels/09/fc/dc/afff211038bfc745722d8d7e846e854e5791968b22c570a530
Successfully built pyfpgrowth


In [ ]:
from google.colab import drive
import pandas as pd
import pyfpgrowth
import time

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
ecommerce = pd.read_csv("/content/drive/MyDrive/Project Skripsi/data.csv")
ecommerce.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom


In [ ]:
ecommerce.dropna(inplace=True)

In [ ]:
ecommerce.drop_duplicates(inplace=True)

In [ ]:
ecommerce.isnull().sum()

TransactionNo    0
Date             0
ProductNo        0
ProductName      0
Price            0
Quantity         0
CustomerNo       0
Country          0
dtype: int64

In [ ]:
ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 531095 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  531095 non-null  object 
 1   Date           531095 non-null  object 
 2   ProductNo      531095 non-null  object 
 3   ProductName    531095 non-null  object 
 4   Price          531095 non-null  float64
 5   Quantity       531095 non-null  int64  
 6   CustomerNo     531095 non-null  float64
 7   Country        531095 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.5+ MB


In [ ]:
ecommerce.drop(['Date', 'ProductNo', 'Price','Quantity','CustomerNo','Country'], axis=1, inplace=True)
ecommerce.head()

,TransactionNo,ProductName
0,581482,Set Of 2 Wooden Market Crates
1,581475,Christmas Star Wish List Chalkboard
2,581475,Storage Tin Vintage Leaf
3,581475,Tree T-Light Holder Willie Winkie
4,581475,Set Of 4 Knick Knack Tins Poppies


In [ ]:
ecommerce_sample = ecommerce[['TransactionNo']].drop_duplicates().sample(n=15000, random_state=42)
ecommerce_sample_df = pd.merge(ecommerce_sample, ecommerce[['TransactionNo', 'ProductName']], on='TransactionNo', how='left')
ecommerce_sample_df.head()

,TransactionNo,ProductName
0,565446,Heart Filigree Dove Large
1,565446,Filigree Heart Bird White
2,565446,Zinc Metal Heart Decoration
3,565446,Peace Wooden Block Letters
4,565446,Vintage Christmas Gift Sack


In [ ]:
ecommerce_sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343308 entries, 0 to 343307
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   TransactionNo  343308 non-null  object
 1   ProductName    343308 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [ ]:
product_count=ecommerce_sample_df[['TransactionNo','ProductName']].groupby('ProductName',as_index=False)['TransactionNo'].count(
    ).sort_values(by='TransactionNo',ascending=False)
product_count.rename(columns = {'TransactionNo':'count'},inplace=True)
product_count

,ProductName,count
815,Cream Hanging Heart T-Light Holder,1542
2620,Regency Cakestand 3 Tier,1455
1656,Jumbo Bag Red Retrospot,1407
2202,Party Bunting,1132
1824,Lunch Bag Red Retrospot,1034
...,...,...
2311,Pink Flock Photo Frame,1
1426,Green/Blue Ceramic Candle Holder,1
1425,Green With Metal Bag Charm,1
2319,Pink Gauze Butterfly Lampshade,1


In [ ]:
association = ecommerce_sample_df.groupby(['TransactionNo'], as_index=True)['ProductName'].apply(lambda x: list(x))

transactions = association.values.tolist()

In [ ]:
min_support = 0.02
min_sup = min_support * len(transactions)
min_confidence = 0.3
min_lift = 5

In [ ]:
start_time = time.time()
patterns = pyfpgrowth.find_frequent_patterns(transactions, min_sup)
rules = pyfpgrowth.generate_association_rules(patterns, min_confidence)
end_time = time.time()
process_time = end_time - start_time
print("Time taken to process:", process_time, "detik")

Time taken to process: 6.769011735916138 detik


In [ ]:
patterns

{('Set Of 12 Mini Loaf Baking Cases',): 300,
 ('Toy Tidy Pink Polkadot',): 301,
 ('Doormat Spotty Home Sweet Home',): 302,
 ('White Wood Garden Plant Ladder',): 303,
 ('Clear Drawer Knob Acrylic Edwardian',): 304,
 ('Red Diner Wall Clock',): 304,
 ('4 Traditional Spinning Tops',): 304,
 ('Jumbo Bag Dolly Girl Design',): 305,
 ('Small Heart Measuring Spoons',): 305,
 ('Alarm Clock Bakelike Orange',): 306,
 ('Set 3 Retrospot Tea/Coffee/Sugar',): 307,
 ('Rotating Silver Angels T-Light Hldr',): 308,
 ('Wooden School Colouring Set',): 308,
 ('Picnic Basket Wicker Large',): 309,
 ('Party Metal Sign',): 310,
 ('Small Marshmallows Pink Bowl',): 310,
 ('Zinc Folkart Sleigh Bells',): 311,
 ('Feltcraft Cushion Owl',): 312,
 ('Toilet Sign Occupied Or Vacant',): 312,
 ('Regency Tea Plate Roses',): 312,
 ('Travel Sewing Kit',): 313,
 ('Fawn Blue Hot Water Bottle',): 313,
 ('Gingham Heart Decoration',): 314,
 ('Hand Warmer Union Jack',): 315,
 ('Scandinavian Reds Ribbons',): 315,
 ('Tea Time Party Bu

In [ ]:
results = []
for rule, confidence in rules.items():
    antecedent = ', '.join(rules[rule][0])
    consequent = rule

    support = patterns[rule] / len(transactions)
    confidence = rules[rule][1]
    lift = confidence / (patterns[rule]/ len(transactions))

    if consequent and len(consequent) > 0:
        consequent = consequent[0]
    else:
        consequent = ''

    if lift >= min_lift and confidence >= min_confidence:
        rows = (antecedent, consequent, support, confidence, lift)
        results.append(rows)

columns = ['Antecedent', 'Consequent', 'Support', 'Confidence','Lift']
associations_dataframe = pd.DataFrame(results, columns=columns)
associations_dataframe

,Antecedent,Consequent,Support,Confidence,Lift
0,Charlotte Bag Suki Design,Red Retrospot Charlotte Bag,0.045733,0.504373,11.028568
1,Red Retrospot Charlotte Bag,Strawberry Charlotte Bag,0.032467,0.673511,20.744701
2,Cream Hanging Heart T-Light Holder,Red Hanging Heart T-Light Holder,0.032733,0.702648,21.465814
3,Gardeners Kneeling Pad Keep Calm,Gardeners Kneeling Pad Cup Of Tea,0.034133,0.697266,20.427704
4,Gardeners Kneeling Pad Cup Of Tea,Gardeners Kneeling Pad Keep Calm,0.040333,0.590083,14.630148
5,Jumbo Bag Red Retrospot,Jumbo Bag Scandinavian Blue Paisley,0.034600,0.678227,19.601947
6,Red Retrospot Charlotte Bag,Charlotte Bag Pink Polkadot,0.034733,0.706334,20.335911
7,Alarm Clock Bakelike Red,Alarm Clock Bakelike Pink,0.035667,0.622430,17.451306
8,Alarm Clock Bakelike Green,Alarm Clock Bakelike Red,0.048133,0.619114,12.862470
9,Regency Cakestand 3 Tier,Roses Regency Teacup And Saucer,0.050800,0.484252,9.532519


In [ ]:
average_Support = associations_dataframe['Support'].mean()
print("Rata-rata Support:", average_Support)

Rata-rata Support: 0.03959540229885057


In [ ]:
average_Confidence = associations_dataframe['Confidence'].mean()
print("Rata-rata confidence:", average_Confidence)

Rata-rata confidence: 0.6457490182990057


In [ ]:
average_Lift = associations_dataframe['Lift'].mean()
print("Rata-rata lift:", average_Lift)

Rata-rata lift: 17.02000797608456
